In [18]:
# Imports
from langchain.prompts import PromptTemplate
from langchain.chains.llm import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings

# Create the embedding function
embedding_function = OpenAIEmbeddings()

# Load the database from existing embeddings
db = Chroma(persist_directory="../chroma-data-langchain-docs", embedding_function=embedding_function)

# Create a prompt with 3 variables: `context`, `question` and `history`
prompt = PromptTemplate(
    template=""""Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

<context>
{context}
</context>

<chat-history>
{history}
</chat-history>

Question: {question}
Helpful Answer:""",
    input_variables=["context", "question", "history"]
)

# Create an LLM with ChatOpenAI
llm = ChatOpenAI(temperature=0)

# Create a chain to reformulate the question based on the chat history
question_generation_prompt = PromptTemplate(
    template="""Based on the following chat history, reformulate the Human's question:
<chat-history>
{history}
Human: {question}
</chat-history>
Relevant question:""",
    input_variables=["history", "question"]
)
generate_question_chain = LLMChain(llm=llm, prompt=question_generation_prompt)

# Create the chain
memory = ConversationBufferMemory(input_key="question")
qa_chain = LLMChain(llm=llm, prompt=prompt, memory=memory)


In [25]:
# A function to chat with the bot.
# This function will first rewrite the question so the chat history is taken into account
# Then it will search for relevant documents based on the reformulated question
# And finally call the qa_chain with the found context documents
def send_message(question):

    # Reformulate the question based on history
    new_question = generate_question_chain({
        "question": question, 
        "history": memory.load_memory_variables({})["history"]
    })
    print('Reformulated question:', new_question["text"], '\n')

    # Query the database as store the results as `context_docs`
    context_docs = db.similarity_search(new_question["text"])

    # Call the chain
    result = qa_chain({"context": "\n".join([d.page_content for d in context_docs]), "question": question})

    # Print the result
    print(result["text"])

In [26]:
send_message('Hi, my name is Emmanuel')

Reformulated question: Can you assist me with something? 

Hello Emmanuel, how can I assist you?


In [21]:
send_message('what is your name?')

Reformulated question: May I know your name?
I am an AI language model, I do not have a name.


In [22]:
send_message('what is my name ?')

Reformulated question: Can you tell me what my name is?
Your name is Emmanuel.
